In [35]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.core import VectorStoreIndex, QueryBundle
from llama_index.llms.ollama import Ollama
from llama_index.core import Document, Settings
from getpass import getpass
from urllib.request import urlopen
import json

In [21]:
ELASTIC_CLOUD_ID = 'My_deployment:YXNpYS1zb3V0aGVhc3QxLmdjcC5lbGFzdGljLWNsb3VkLmNvbSQyMDFiMjBmMjIwZmQ0NjQyYTE4YWQzNWYxMzAyMWZlNSRiMDlhNTcyOWU5YmE0MGJkYjVhMjdmYzAxZjhiMzI2Mg=='
ELASTIC_API_KEY = 'Um5jU0FKVUJLcWtQQjJ6NzRNa2Q6MzhwRzNIaHVTdXVIOGZVSm16TElGQQ=='

In [36]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/workplace-documents.json"

response = urlopen(url)
workplace_docs = json.loads(response.read())

# Building Document required by LlamaIndex.
documents = [
    Document(
        text=doc["content"],
        metadata={
            "name": doc["name"],
            "summary": doc["summary"],
            "rolePermissions": doc["rolePermissions"],
        },
    )
    for doc in workplace_docs
]

In [29]:
es_vector_store = ElasticsearchStore(
    index_name="workplace_index",
    vector_field="content_vector",
    text_field="content",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)
# Embedding Model to do local embedding using Ollama.
ollama_embedding = OllamaEmbedding("llama3.1:latest")
# LlamaIndex Pipeline configured to take care of chunking, embedding
# and storing the embeddings in the vector store.
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=100),
        ollama_embedding,
    ],
    vector_store=es_vector_store,
)

In [30]:
pipeline.run(show_progress=True, documents=documents)

Generating embeddings: 100%|██████████| 27/27 [00:09<00:00,  2.96it/s]


[TextNode(id_='f27dfd73-ab24-4313-9092-707198b5e664', embedding=[-0.7400327324867249, -5.215385437011719, -2.2583889961242676, -0.421898752450943, 0.5727839469909668, 2.003343343734741, -1.3359954357147217, -0.348315566778183, -1.2860417366027832, -1.6182689666748047, 3.1957571506500244, -2.5199577808380127, 3.7962119579315186, -0.20742912590503693, -2.478010892868042, -0.5365419387817383, 0.35985812544822693, -1.2815645933151245, 2.3197410106658936, -1.4789936542510986, -0.792087733745575, -0.7422369718551636, -0.2114732712507248, -3.7011735439300537, -0.7118976712226868, 0.4434424340724945, 0.45786193013191223, -6.265675067901611, -2.0846989154815674, -0.7955536246299744, -1.6012296676635742, 1.990424394607544, 1.195737361907959, 1.2831668853759766, 0.920696496963501, -1.9089969396591187, -0.4732897877693176, 0.9133491516113281, 3.584474563598633, 0.19242523610591888, 0.4294631779193878, 1.5223119258880615, -0.4477807879447937, -0.5973471403121948, 2.5910918712615967, -2.491375446319

In [31]:
Settings.embed_model = ollama_embedding
local_llm = Ollama(model="llama3.1:latest")

In [32]:
index = VectorStoreIndex.from_vector_store(es_vector_store)
query_engine = index.as_query_engine(local_llm, similarity_top_k=10)

# Customer Query
query = "What are the organizations sales goals?"
bundle = QueryBundle(
    query_str=query, embedding=Settings.embed_model.get_query_embedding(query=query)
)

response = query_engine.query(bundle)

print(response.response)

The company's sales teams are responsible for promoting and selling the company's products and services to potential clients. They aim to establish relationships with customers, understand their needs, and ensure that the offered solutions align with their requirements. The organization is divided into four regions (The Americas, Europe, Asia-Pacific, and Middle East & Africa), each led by an Area Vice-President who oversees dedicated account managers, sales representatives, and support staff.
